# 导入库

In [8]:
from osgeo import gdal, osr
import numpy as np
import matplotlib.pyplot as plt
import os #系统库

# 拼接数据函数

In [2]:
def raster_merge(image_list, xres, yres, merged_path):
    """
    :param image_list: 拼接文件路径列表，可同时拼接多个文件
    :param xres: 输出文件水平分辨率
    :param yres: 输出文件垂直方向分辨率，一般为负值
    :param merged_name: 输出文件路径
    :param: nodata: 设置输出图像的无效值
    :return:
    """
    vrt = gdal.BuildVRT("merged.vrt", image_list)
    gdal.Translate(merged_path, vrt, xRes=xres, yRes=yres, resampleAlg="bilinear", outputType=gdal.GDT_Int16,
                   creationOptions=["COMPRESS=LZW"])
    vrt = None
    os.remove("merged.vrt")

# 读取DEM数据，并拼接

In [3]:
#存储DEM数据的文件夹
input_folder = "E:/l3/极端事件汇报/代码和文件/3.变换DEM数据/输入文件/DEM数据/"
#存储DEM数据路径的列表
files = [input_folder + f for f in os.listdir(input_folder) if f[-4:]==".tif"]
merged_dem = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\merged_dem.tif"
raster_merge(files, 0.00027777777777777805, 0.0002777777777777774, merged_dem)

# 使用延庆区的shapefile裁剪DEM数据

In [4]:
#拼接的DEM路径
dem_merged = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\merged_dem.tif"
in_mask = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\区域文件\yanqing.shp"
yanqing_dem = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\yanqing.tif"
ds = gdal.Warp(yanqing_dem, dem_merged
                     , cutlineDSName=in_mask, cropToCutline=True,
                     dstNodata=-32768, creationOptions=[ "COMPRESS=LZW"])
del ds

# 定义Alberts投影函数

In [13]:
def Albers_Conic_Equal_Area_China(input_tif, out_path):
    #使用PROJ字符串进行初始化
    dataset = gdal.Open(input_tif)
    # proj = dataset.GetProjection()
    srs = osr.SpatialReference()
    # srs.ImportFromWkt(proj)
    # print(srs.ExportToproj4())
    # print('---------------------------')
    
    #定义投影
    srs.ImportFromProj4("+proj=aea +lat_0=0 +lon_0=105 +lat_1=25 +lat_2=47 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")
    repro_ds = gdal.Warp(out_path, dataset, dstSRS=srs,
                           xRes=30, yRes=30,
                           resampleAlg = gdal.GRA_Bilinear,
                           outputType=gdal.GDT_Int16,
                           options=["COMPRESS=LZW"])
    # srcNodata=3, dstNodata=3,
                           # outputBounds=(542103.969791774, 4337959.14804815, 1122333.96979177, 4848349.14804815),
    print("projection has been processed.")
    del dataset
    del repro_ds

# 读取DEM数据，转换坐标系

In [14]:
# 延庆DEM数据路径
dem_path = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\yanqing.tif"
output_path = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\yanqing_reproj.tif"
Albers_Conic_Equal_Area_China(dem_path, output_path)

projection has been processed.


# 将投影后的DEM分辨率重采样到100 m

In [15]:
dem_path = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\yanqing_reproj.tif"
output_path = r"E:\l3\极端事件汇报\代码和文件\3.变换DEM数据\输入文件\输出数据\yanqing_reproj_resampled.tif"

resampled_ds = gdal.Warp(output_path, dem_path, 
                           xRes=100, yRes=100,
                           resampleAlg = gdal.GRA_Bilinear,
                           options=["COMPRESS=LZW"])
del resampled_ds